## Graph Transformers for Blood-Brain-Barrier Penetration Prediction
**Ayush Noori**

First, I load the relevant libraries.

In [1]:
# import base libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # inline plots
%matplotlib inline

# TDC library
from tdc.benchmark_group import admet_group
from tdc.chem_utils import MolConvert

# XGBoost
import xgboost as xgb

# import Optuna
import optuna
from optuna.samplers import TPESampler
import optuna.visualization.matplotlib as oviz

# logging to show Optuna output
import logging
import sys

# time management
from datetime import datetime

# create time object used for file names
my_time = datetime.now()

In [2]:
group = admet_group(path = 'data/')
predictions_list = []

Found local copy...


Next, I retrieve the data.

In [3]:
# set seed and benchmark in benchmark group
seed = 1
benchmark = group.get('BBB_Martins') 
    
# all benchmark names in a benchmark group are stored in group.dataset_names
predictions = {}
name = benchmark['name']
train_val, test = benchmark['train_val'], benchmark['test']
# train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)

I define the XGBoost training function. First, I convert the SMILES structures to fingerprints. Note that conversion from `SMILES` to `RDKit2D` requires `pip install git+https://github.com/bp-kelley/descriptastorus` and `pip install pandas-flavor`.

The parameter definition code was inspired by [this Medium post](https://medium.com/optuna/using-optuna-to-optimize-xgboost-hyperparameters-63bfcdfd3407) with [this source code](https://gist.github.com/Crissman/4ddeec6718627ecef46f863e1bf90424#file-xgboost_integration-py). See [the documentation](https://xgboost.readthedocs.io/en/latest/parameter.html) for a full list of possible parameters.

In [14]:
# define training function
def train_xgboost(trial):

    # define fingerprint
    mol_dst = trial.suggest_categorical("mol_dst", ["ECFP2", "ECFP4", "MACCS", "Morgan", "Daylight", "RDKit2D"])

    # define the XGBoost parameters, inspired by 
    params = {
            "objective": "binary:logistic",
            "eval_metric": "auc",
            "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
            "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
            "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
        }

    if params["booster"] == "gbtree" or params["booster"] == "dart":
        params["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        params["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
        params["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
        params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if params["booster"] == "dart":
        params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        params["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
        params["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)

    # print all of the hyperparameters of the training iteration:
    print("\n===== TRIAL #{} =====".Format(trial.number))
    print("Fingerprint: {}".format(mol_dst))

    # convert fingerprint
    converter = MolConvert(src = 'SMILES', dst = mol_dst)
    train_val_features = pd.DataFrame([converter(x) for x in train_val['Drug']])

    # define the optimized DMatrix object
    dtrain_val = xgb.DMatrix(train_val_features, label = train_val['Y'])

    # perform cross-validation
    cv_results = xgb.cv(dtrain = dtrain_val, params = params, nfold = 5, \
        num_boost_round = 100, early_stopping_rounds = 10, metrics = "auc", as_pandas = True, seed = seed)

    # train final model based on parameters in Optuna trial
    xgb_model = xgb.train(dtrain = dtrain_val, params = params, num_boost_round = 50)

    # report the final boosting metric as the summary metric for this Optuna trial
    test_auc_mean = cv_results["test-auc-mean"].tail(1)

    # return model and metric
    return xgb_model, test_auc_mean

I define the Optuna objective function.

In [11]:
# define directories
optuna_dir = "optuna/"
model_dir = optuna_dir + "models/"
study_dir = optuna_dir + "database/"

# define objective function
def objective(trial):

    # start the training loop
    trial_xgb_model, trial_test_auc_mean = train_xgboost(trial)

    # save model for this loop
    trial_xgb_model.save_model(model_dir + "xgboost_model_{}.json".format(trial.number))

    return trial_test_auc_mean

I run the Optuna trials.

In [9]:
# add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

# create study
study_name = "xgboost-study"  # unique identifier of the study
storage_name = "sqlite:///{}.db".format(study_dir + study_name)
study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed = 1234, multivariate = True), study_name = study_name, storage = storage_name, load_if_exists = False)

# optimize hyperparameters
study.optimize(objective, n_trials = 20, gc_after_trial = True)

[I 2022-05-04 15:16:53,380] A new study created in RDB with name: xgboost-study


A new study created in RDB with name: xgboost-study
A new study created in RDB with name: xgboost-study


[I 2022-05-04 15:17:07,307] Trial 0 finished with value: 0.8368359736969764 and parameters: {'mol_dst': 'MACCS', 'booster': 'dart', 'lambda': 0.4625032932437533, 'alpha': 0.10173281877564906, 'max_depth': 4, 'eta': 0.00010184999316414055, 'gamma': 0.0029356446287146696, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.11511436577603909, 'skip_drop': 8.300186889879715e-06}. Best is trial 0 with value: 0.8368359736969764.


Trial 0 finished with value: 0.8368359736969764 and parameters: {'mol_dst': 'MACCS', 'booster': 'dart', 'lambda': 0.4625032932437533, 'alpha': 0.10173281877564906, 'max_depth': 4, 'eta': 0.00010184999316414055, 'gamma': 0.0029356446287146696, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.11511436577603909, 'skip_drop': 8.300186889879715e-06}. Best is trial 0 with value: 0.8368359736969764.
Trial 0 finished with value: 0.8368359736969764 and parameters: {'mol_dst': 'MACCS', 'booster': 'dart', 'lambda': 0.4625032932437533, 'alpha': 0.10173281877564906, 'max_depth': 4, 'eta': 0.00010184999316414055, 'gamma': 0.0029356446287146696, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.11511436577603909, 'skip_drop': 8.300186889879715e-06}. Best is trial 0 with value: 0.8368359736969764.


[I 2022-05-04 15:17:21,712] Trial 1 finished with value: 0.6761550354134462 and parameters: {'mol_dst': 'MACCS', 'booster': 'gblinear', 'lambda': 0.0003505816825467768, 'alpha': 0.08974682978025321}. Best is trial 0 with value: 0.8368359736969764.


...

[I 2022-05-04 15:17:40,745] Trial 2 finished with value: 0.8812758563998226 and parameters: {'mol_dst': 'ECFP4', 'booster': 'gblinear', 'lambda': 0.18816015917069162, 'alpha': 3.009357492612275e-08}. Best is trial 2 with value: 0.8812758563998226.


Trial 2 finished with value: 0.8812758563998226 and parameters: {'mol_dst': 'ECFP4', 'booster': 'gblinear', 'lambda': 0.18816015917069162, 'alpha': 3.009357492612275e-08}. Best is trial 2 with value: 0.8812758563998226.
Trial 2 finished with value: 0.8812758563998226 and parameters: {'mol_dst': 'ECFP4', 'booster': 'gblinear', 'lambda': 0.18816015917069162, 'alpha': 3.009357492612275e-08}. Best is trial 2 with value: 0.8812758563998226.


[W 2022-05-04 15:17:41,112] Trial 3 failed because of the following error: NameError("free variable 'f2' referenced before assignment in enclosing scope")
Traceback (most recent call last):
  File "C:\Users\unity\.conda\envs\gnn\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\unity\AppData\Local\Temp/ipykernel_26844/4271217205.py", line 10, in objective
    trial_xgb_model, trial_test_auc_mean = train_xgboost(trial)
  File "C:\Users\unity\AppData\Local\Temp/ipykernel_26844/2933544791.py", line 9, in train_xgboost
    train_val_features = pd.DataFrame([converter(x) for x in train_val['Drug']])
  File "C:\Users\unity\AppData\Local\Temp/ipykernel_26844/2933544791.py", line 9, in <listcomp>
    train_val_features = pd.DataFrame([converter(x) for x in train_val['Drug']])
  File "C:\Users\unity\.conda\envs\gnn\lib\site-packages\tdc\chem_utils\featurize\molconvert.py", line 802, in __call__
    return self.func(x)
  File

Trial 3 failed because of the following error: NameError("free variable 'f2' referenced before assignment in enclosing scope")
Traceback (most recent call last):
  File "C:\Users\unity\.conda\envs\gnn\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\unity\AppData\Local\Temp/ipykernel_26844/4271217205.py", line 10, in objective
    trial_xgb_model, trial_test_auc_mean = train_xgboost(trial)
  File "C:\Users\unity\AppData\Local\Temp/ipykernel_26844/2933544791.py", line 9, in train_xgboost
    train_val_features = pd.DataFrame([converter(x) for x in train_val['Drug']])
  File "C:\Users\unity\AppData\Local\Temp/ipykernel_26844/2933544791.py", line 9, in <listcomp>
    train_val_features = pd.DataFrame([converter(x) for x in train_val['Drug']])
  File "C:\Users\unity\.conda\envs\gnn\lib\site-packages\tdc\chem_utils\featurize\molconvert.py", line 802, in __call__
    return self.func(x)
  File "C:\Users\unity\.conda\envs

NameError: free variable 'f2' referenced before assignment in enclosing scope

After training, I output Optuna trial results.

In [ ]:
# get pruned and complete trials
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

# print print study statistics
print("\nStudy Statistics:")
print("- Finished Trials: ", len(study.trials))
print("- Pruned Trials: ", len(pruned_trials))
print("- Complete Trials: ", len(complete_trials))

print("\nBest Trial:")
best_trial = study.best_trial
print("- Number: ", best_trial.number)
print("- Value: ", best_trial.value)
print("- Hyperparameters: ")
for key, value in best_trial.params.items():
    print("   - {}: {}".format(key, value))

# save and view output
study_results = study.trials_dataframe(attrs=("number", "value", "params", "state"))
study_results.to_csv(optuna_dir + "{}.{}_{}.{}.{}_OptunaHistory.csv".format(my_time.hour, my_time.minute, my_time.month, my_time.day, my_time.year))

I visualize the Optuna trial iterations.

In [ ]:
v1 = oviz.plot_param_importances(study)
v2 = oviz.plot_optimization_history(study)
v3 = oviz.plot_slice(study)

def fig_name(name):
    return(optuna_dir + "output/" + "{}.{}_{}.{}.{}_{}.pdf".format(my_time.hour, my_time.minute, my_time.month, my_time.day, my_time.year, name))

v1.figure.savefig(fig_name("HyperparameterImportance"))
v2.figure.savefig(fig_name("OptimizationHistory"))

Finally, I train a new model based on the best trial.

In [ ]:
# train new model based on the best trial
best_xgb_model = xgb.Booster()
best_xgb_model.load_model(model_dir + "xgboost_model_{}.json".format(best_trial.number))

Now, I evaluate on the independent test set.

In [ ]:
# convert fingerprint
test_converter = MolConvert(src = 'SMILES', dst = best_trial.params['mol_dst'])
test_features = pd.DataFrame([test_converter(x) for x in test['Drug']])
dtest = xgb.DMatrix(test_features, label = test['Y'])

# evaluate on test set
y_pred_test = best_xgb_model.predict(dtest)
plt.hist(y_pred_test)

# append predictions to test set object
predictions[name] = y_pred_test
predictions_list.append(predictions)